In [1]:
import numpy as np
import pandas as pd
import glob

(preprocessing cpi data)

In [2]:
df = pd.read_csv('cpi.csv', converters = {'TERMINATED': lambda x: 0 if x == '' else 1}, low_memory = False) 
df.head()

,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1914-01,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.200,6.0,NaN,NaN,0,1
1,1914-01,Canada,2016A000011124,All-items (1992=100),1992=100,7,units,0,v41713403,2.309,7.2,NaN,NaN,1,1
2,1914-01,Canada,2016A000011124,Goods and services,2002=100,17,units,0,v41691221,2.273,6.0,NaN,NaN,1,1
3,1914-02,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.200,6.0,NaN,NaN,0,1
4,1914-02,Canada,2016A000011124,All-items (1992=100),1992=100,7,units,0,v41713403,2.309,7.2,NaN,NaN,1,1


In [3]:
use = df.drop(columns = ['STATUS', 'SYMBOL', 'SCALAR_FACTOR', 'SCALAR_ID', 'DECIMALS'])
# df.head()

use = use[(use['REF_DATE'] > '2000-01') & (use['UOM'] > '1992=100')
          & (use['TERMINATED'] == 0)]
# df.drop(unuse, inplace = True)
use.head()

# list(df.columns)
# df.shape
# use.to_csv('tempout.csv', sep = ',')

,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,VECTOR,COORDINATE,VALUE,TERMINATED
503731,2000-02,Canada,2016A000011124,All-items,2002=100,17,v41690973,2.2,94.1,0
503732,2000-02,Canada,2016A000011124,Food,2002=100,17,v41690974,2.3,91.9,0
503733,2000-02,Canada,2016A000011124,Food purchased from stores,2002=100,17,v41690975,2.4,91.4,0
503734,2000-02,Canada,2016A000011124,Meat,2002=100,17,v41690976,2.5,87.9,0
503735,2000-02,Canada,2016A000011124,Fresh or frozen meat (excluding poultry),2002=100,17,v41690977,2.6,82.9,0


In [4]:
# include canada(??)
reg = ['Ontario', 'Toronto, Ontario']
use = use[use['GEO'].isin(reg)]
use.head()
use.to_csv('cleancpi1.csv')

In [5]:
vfeats = use[['Products and product groups', 'VECTOR', 'GEO']].drop_duplicates()
vfeats.to_csv('fnames.csv')

In [6]:
# final = pd.read_csv('cleancpi1.csv')
to = use.drop(columns = ['UOM', 'UOM_ID', 'COORDINATE', 'TERMINATED', 'DGUID'])

# to.to_csv('cleancpi2.csv')


In [11]:
dto = to.drop(columns = ['GEO', 'Products and product groups'])

dkp = ['v41691920', 'v41692890', 'v41692892', 'v41692001', 'v41692002', 'v41692000', 'v41693222', 'v41692027','v41692004', 'v41691968']
dkpt = dto[dto['VECTOR'].isin(dkp)]
dkpt = dkpt[dkpt['REF_DATE'] > '2002-11']
dkpt['REF_DATE'] = pd.to_datetime(dkpt['REF_DATE'], yearfirst= True)
# dtp = dto.pivot(index = 'REF_DATE', columns = 'VECTOR')
dfkpt = dkpt.pivot(index = 'REF_DATE', columns = 'VECTOR')

dfkpt

dfkpt.to_csv('curdf1.csv')

# dtp.to_csv('fvals.csv')
# v1563916460


keep = vfeats[vfeats['VECTOR'].isin(dkp)]
keep



,Products and product groups,VECTOR,GEO
504732,Food,v41691920,Ontario
504780,Telephone services,v41691968,Ontario
504812,Local and commuter transportation,v41692000,Ontario
504813,City bus and subway transportation,v41692001,Ontario
504814,Taxi and other local and commuter transportati...,v41692002,Ontario
504816,Health and personal care,v41692004,Ontario
504839,"Alcoholic beverages, tobacco products and recr...",v41692027,Ontario
504878,Rented accommodation,v41692890,"Toronto, Ontario"
504880,"Water, fuel and electricity",v41692892,"Toronto, Ontario"
573077,Internet access services (200212=100),v41693222,Ontario


(preprocessing toronto rent data)

In [ ]:
path = r'C:\sashack'
all_files = glob.glob(path +"/rent*.csv")
flst = []

for file in all_files:
    f = pd.read_csv(file, index_col = None)
    flst.append(f)

rents = pd.concat(flst, axis = 0, ignore_index=True)
# rents.to_csv('avgrents.csv')
rents.head()

In [ ]:
# areas = set(rents['Area'])
# print(areas)
exclude = ['Port Credit', 'Uxbridge', 'Bronte', 'Richmond Hill/Vaughan/King', 'Brampton (West)', 'Georgina', 
           'Bradford/West Gwillimbury/New Tecumseth', 'Malton', 'Ajax/Pickering', 'East Gwillimbury/Newmarket','Markham', 
          'Aurora', 'Mississauga Centre/Streetsville', 'Milton/Halton Hills', 'Orangeville/Mono', 'Whitchurch/Stouffville',
          'Oakville (excl. Bronte)', 'Churchill Meadows/Erin Mills', 'Cooksville', 'Brampton (East)', 'Caledon']
rents = rents[~rents['Area'].isin(exclude)]
rents.head()
rents.to_csv('torrentswna.csv')
# rents.head()
# rents.dropna(subset=['bqual', 'onequal', 'twoqual', 'thrqual', 'totqual'])

# rents.head()

trents = rents.dropna(subset=['bqual', 'onequal', 'twoqual', 'thrqual', 'totqual'], how = 'all')
trents.head()
torents = trents.sort_values(by = ['Area'])
# trents.head()
torents.to_csv('torents.csv')

time series analysis for multvar(?)